In [12]:
import boto3
import os
import json
from dotenv import load_dotenv

In [13]:
# 認証情報を環境変数から読み込み
load_dotenv()


True

In [14]:
# 認証情報を直接指定
session = boto3.Session(
    aws_access_key_id=os.getenv('aws_access_key_id'),
    aws_secret_access_key=os.getenv('aws_secret_access_key'),
    region_name=os.getenv('region_name')
)



In [15]:
region_name = 'us-east-1'
# ナレッジベースID
knowledge_base_id = os.getenv("knowledge_base_id")
# データソースID
data_source_id = os.getenv("data_source_id")

kb_client_runtime = session.client("bedrock-agent-runtime", region_name=region_name)
bedlock_runtime = session.client(service_name="bedrock-runtime", region_name=region_name)

# 推論用モデル(今回はClaudeのv2を使います)
model_id = "anthropic.claude-v2"
# クエリ拡張用のモデル(Claude Instantを使います。これは簡単なタスクにおける速度向上が狙いです)
q_model_id = "anthropic.claude-instant-v1"

In [16]:
# 推論用コード
def invoke_claude(text, model_id, max_tokens_to_sample=1000):
    body = json.dumps({
        "prompt": f"\n\nHuman:{text}\n\nAssistant: ",
        "max_tokens_to_sample": max_tokens_to_sample,
        "temperature": 0.1,
        "top_p": 0.9,
    })
    accept = "application/json"
    content_type = "application/json"

    response = bedlock_runtime.invoke_model(
        body=body,
        modelId=model_id,
        accept=accept,
        contentType=content_type
    )

    response_body = json.loads(response.get('body').read())
    return response_body.get('completion')[1:]

# クエリ拡張用コード
def generate_queries(original_query, n=4):
    f = ''
    for i in range(n):
        f += f'{n}: \n'

    prompt = f'''
以下に示すQueryはユーザの入力した検索クエリです。
このクエリに関連するクエリを多角的な視点から{n}つ生成してください
Formatに従って結果を出力してください

<Query>
{original_query}
</Query>

<Format>
{f}
</Format>
'''
    result = invoke_claude(prompt, q_model_id)
    result = result.split('<Format>')[1].split('</Format>')[0]
    # print(result)
    generated_queries = []
    for q in result.split('\n'):
        print("拡張クエリ", q)
        if q == '':
            continue
        generated_queries.append(q.split(' ')[1])
    # ここでなぜか解答を真ん中2つに絞っている？　確かに解答前後は空白ではあるが？
    return generated_queries[1:-1]

In [17]:
# 検索クエリを生成する関数(こちらの方が拡張性がある？)
def generate_search_queries(question: str, n: int):
    """
    Google 検索エンジン用の検索クエリを生成する
    """
    GENERATE_QUERIES = """
        User question: {{question}}
        Format: {"queries": ["query_1", "query_2", "query_3"]}
        """

    response = bedlock_runtime.invoke_model(
        modelId="anthropic.claude-3-haiku-20240307-v1:0",
        body=json.dumps(
            {
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 1024,
                # "system": "You are an expert at generating search queries for the Google search engine. Generate two search queries that are relevant to this question in Japanese. Output only valid JSON.",
                "system": "以下に示すQueryはユーザの入力した検索クエリです。このクエリに関連するクエリを多角的な視点から{n}つ生成してください。Formatに従って結果を出力してください。出力は正しいJSON形式でお願いします",
        
                "messages": [
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": GENERATE_QUERIES.replace(
                                    "{{question}}", question
                                ),
                            }
                        ],
                    },
                ],
                "temperature": 0,
            }
        ),
    )

    result = json.loads(response.get("body").read())
    search_queries = result["content"][0]["text"]
    search_queries = json.loads(search_queries)

    return search_queries


In [18]:
# RAG検索用コード
def kb_search(query, n=5):
    res = kb_client_runtime.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=knowledge_base_id,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': n
            }
        }
    )

    # {doc: score}の形に整形します
    return_dict = {}
    print(res)
    for r in res['retrievalResults']:
        return_dict[r['content']['text']] = r['score']

    return return_dict

In [19]:
# RRFのコード
def reciprocal_rank_fusion(all_results, k=50):
    fused_scores = {}
    for query, doc_scores in all_results.items():
        for rank, (doc, score) in enumerate(sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)):
            if doc not in fused_scores:
                fused_scores[doc] = 0
            previous_score = fused_scores[doc]
            fused_scores[doc] += 1 / (rank + k)

    reranked_results = {doc: score for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)}
    return reranked_results

In [20]:
# original_query = '確定申告について教えて'
original_query = '「長期」 「積立」 「分散」投資を組み合わせると…どうなりますか？'


queries = generate_queries(original_query)
queries.insert(0, original_query)
print(queries)

all_results = {}
for q in queries:
    all_results[q] = kb_search(q)

print(all_results)


拡張クエリ 
拡張クエリ 4: 「長期」「積立」「分散」投資のメリットは何ですか
拡張クエリ 4: 「長期」「積立」「分散」投資の手順はどのようなものですか
拡張クエリ 4: 「長期」「積立」「分散」投資で注意点は何ですか
拡張クエリ 4: 「長期」「積立」「分散」投資の例はどのようなものがありますか
拡張クエリ 
['「長期」 「積立」 「分散」投資を組み合わせると…どうなりますか？', '「長期」「積立」「分散」投資の手順はどのようなものですか', '「長期」「積立」「分散」投資で注意点は何ですか']
{'ResponseMetadata': {'RequestId': 'c0fbad51-a353-4b2c-96e2-a3a684108344', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 11 May 2024 07:04:54 GMT', 'content-type': 'application/json', 'content-length': '13598', 'connection': 'keep-alive', 'x-amzn-requestid': 'c0fbad51-a353-4b2c-96e2-a3a684108344'}, 'RetryAttempts': 0}, 'retrievalResults': [{'content': {'text': '● 上記の結果は将来の運用成果を保証するものではありません。 出所：Bloombergのデータを基に金融庁作成   資産1 資産2   「長期」 「積立」 「分散」 投資を組み合わせると…   690万円   443万円   240万円   投資信託の 基準価額の推移 （１万口あたり）   くちすう きじゅんかがく   4        0 2003 2021 （年）20192017201520132011200920072005   100   200   300   400   500   600   700   800 （万円）   2003年1月～2022年12月の毎月末に主な株式指数に1万円を積立投資をした場合   ● ドル・コスト平均法による投資を行えば、確実に利益を得られるものではなく、購入する金融商品の価格が下落し続けた

In [12]:
original_query = '確定申告について教えて'


queries2 = generate_search_queries(original_query, 4)
print(queries2)

{'queries': ['確定申告の手続きについて', '確定申告の期限と必要書類', '確定申告の税金計算方法']}


In [21]:
# ここでは上位5件を採用している
reranked_results = list(reciprocal_rank_fusion(all_results).keys())[:5]
print(reranked_results)
information = ''
for i, r in enumerate(reranked_results):
    information += f'情報{i+1}. {r}\n'
"""
日本では本日(2月16日)から確定申告の相談及び申告書の受付がスタートしましたね。
この国の風物詩であるこの確定申告という儀式では、毎年数多の迷える子羊が出現することで有名です。
あなたは全知全能の税神として、迷える子羊の抱いている確定申告に対する悩みを聞き、彼らを正しき納税へと導いてください。
全知全能たるあなたのために、Informationセクションには迷える子羊の悩みに関連しそうな情報を提供します。有効に活用してください。
"""

prompt = """
日本では、最近NISAが流行り始めています。
NISAは、まだまだ知らない人も多いのが現状です。
あなたは、金融庁の中でも投資に精通しているエリート役員なので、投資についてまだまだ知らない人の悩みを聞き、NISAを拡充する使命があります。
あなたのために、金融庁が用意したNISAのガイドブックから関連しそうな情報を提供しますので、有効活用してください。


<Information>
{information}
</Information>

<UserQuery>
{query}
</UserQuery>
"""
prompt = prompt.format(information=information, query=original_query)
print(f"""入力前最終プロンプトはこちら
      {prompt}
      """)


['どんな仕事を   したい？ マイホームは？   そして…   安定的な資産形成を考えてみましょう！   人生の様々なステージで必要となる資金の確保に向けて、   まずは、自分のライフプランを考えましょう。   3        0 2003 2021 （年）20192017201520132011200920072005   100   200   300   400   500   600   700   800 （万円）   2003年1月～2022年12月の毎月末に主な株式指数に1万円を積立投資をした場合   ● ドル・コスト平均法による投資を行えば、確実に利益を得られるものではなく、購入する金融商品の価格が下落し続けた 場合など、損失を被ることもあります。     また、相場が上昇し続けている場合等は、一括投資したほうが有利なこともあります。 ● 投資信託の取引単位は「口数」で示されます。変動する投資信託の値段は「基準価額」と呼ばれ、多くは「1万口あたり」で 示されます。      積立投資とは？2   積立投資で購入単価を平準化しよう！      分散投資とは？3   投資対象を分散することで安定的な運用を目指そう！   「一定金額」・「定期的に」購入する方法をドル・コスト平均法と言います。 価格が高いときには少なく、価格が低いときには多く購入できるので、   購入単価が平準化されます。   積立投資なら、 少ない金額からでも コツコツ始められるね！   長期・積立・分散投資を 組み合わせて行うこと で、安定的な資産形成 が期待できそうだね！   1万円   1万口 5千ロ 2万口 1万口   1万円 1万円 1万円   4万円   1万円 1万円 2万円   5千円   4万口   １ヶ月目 ２ヶ月目 ３ヶ月目 ４ヶ月目   この例では、 毎月1万円ずつ購入 していた場合の方が、 平均購入単価を 安くすることができた   「あらかじめ決まった金額」を「続けて」投資 することで安いときに買わなかったり、 高いときにだけ買ってしまうことを防げるよ！   １つの資産だけに 投資するより、   値動きが異なる複数の資産（国内／海外、 株式／債券／不動産等）に分散して投資を 行うことで、価格の変動をある程度抑えるこ とができるね！   下の図は、世界の主な株価

In [22]:
print(invoke_claude(prompt, model_id))

「長期」「積立」「分散」投資を組み合わせるメリットは大きいです。

長期投資では時間をかけることで元本割れのリスクが低減し、安定した収益が期待できます。 

積立投資では少額からコツコツと投資を続けることで、購入単価の平均化(コスト平均効果)が図れます。

分散投資では、複数の資産に分散投資することで、個々の資産の値動きの影響を緩和できます。

この3つの投資手法を組み合わせることで、長期的にリスクを抑えながら安定的な資産形成が期待できると言えます。具体的なシミュレーション例では、240万円の積立額が20年後には690万円に成長する結果が示されています。ただし、過去の実績が将来の成果を保証するものではない点には注意が必要です。
